#### Vamos carregar nossa base de credito atraves do panda novamente


In [1]:
import pandas as pd
base = pd.read_csv('credito.csv')

In [2]:
base.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


#### Pra aproveitarmos o tratamento de valores incosistentes que ja foi dado no caderno 1, vamos colocar aqui as linhas de codigo pra atribuir o valor da media (sem as idades negativas) para as idades negativas

In [3]:
base.loc[base.age < 0, 'age'] = 40.92

In [4]:
base.loc[base.age < 0]

,clientid,income,age,loan,default


In [5]:
base.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.927689,4444.369695,0.141500
std,577.494589,14326.327119,13.261825,3045.410024,0.348624
min,1.000000,20014.489470,18.055189,1.377630,0.000000
25%,500.750000,32796.459717,29.072097,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


#### Agora vamos comecar a tratar os valores faltantes. O comando a seguir nos retorna true caso alguma idade não esteja preenchida 

In [6]:
pd.isnull(base['age'])

0       False
1       False
2       False
3       False
4       False
        ...  
1995    False
1996    False
1997    False
1998    False
1999    False
Name: age, Length: 2000, dtype: bool

#### Uma visualizacao parcial pode nos enganar, mas observe que o comando a seguir nos informa que falta o preenchimento de 3 idades, alem disso 


In [7]:
base.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.927689,4444.369695,0.141500
std,577.494589,14326.327119,13.261825,3045.410024,0.348624
min,1.000000,20014.489470,18.055189,1.377630,0.000000
25%,500.750000,32796.459717,29.072097,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


#### Podemos refinar o comando ja utilizado, colocando ele dentro de um loc, a fim de encontrar o que queremos

In [8]:
base.loc[pd.isnull(base['age'])]

,clientid,income,age,loan,default
28,29,59417.805406,NaN,2082.625938,0
30,31,48528.852796,NaN,6155.784670,0
31,32,23526.302555,NaN,2862.010139,0


#### Na nosa base podemos separar nossos atributos (no nosso caso as colunas) em dois tipos, previsores e classe. Pra utilizar algoritmos de aprendizagem de maquina no futuro e necessario fazer essa divisao, entao vamos a criacao dessas variaveis

#### Como o id da pessoa e uma informacao que nao reflete o resutado (se paga o emprestimo ou nao), por isso ela nao vai em nenhuma variavel 


In [9]:
# vamos capturar todas linhas, e nas colunas vamos pegar da 1 a 3 inclusa deixando a coluna 0 e 4 de fora
previsores = base.iloc[:,1:4].values
# agora vamos caputar so a coluna default
classe = base.iloc[:,4].values


#### Os proximos passos me colocaram em contato com uma coisa interessante, o material que estou estudando fazia um 
### from sklearn.preprocessing import Imputer
#### Destaco isso pois ao tentar executar deu erro, e ai fui pesquisar e vi que teve uma atualizacao, no qual teve mudanca de nome, dessa forma tive que ir na documentacao mais atual, ver os exemplos e adaptar em relacao ao que eu estava estudando. Entao caso se depare com isso saiba que sempre e bom ir na documentacao pra entender como as coisas funcionam

In [10]:
from sklearn.impute import SimpleImputer


In [11]:
import numpy as np

In [12]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')

In [13]:
imputer = imputer.fit(previsores[:,0:3])

In [14]:
previsores[:,0:3] = imputer.fit_transform(previsores[:,0:3])

#### Note que estamos preenchendo os valores na variavel previsores e nao na nossa base. Pra ver que nao temos mais nan podemos usar o codigo a seguir, que soma os valores dos previsores e ve se essa soma tem algum nan.Como nao vai ter o resultado sera falso

In [15]:
soma_nan = np.sum(previsores)
array_tem_nan = np.isnan(soma_nan)

print(array_tem_nan)


False


# Escalonamento de atributos – base crédito


#### Vamos agora mudar a escala da tabela previsores, pois como os atributos estao em escalas diferentes, podemos ter resultados influenciados quando aplicarmos algoritmos que usam distancia euclidiana para analisar, pois vao dar importancia maior a um atributo em relacao ao outro por conta disso

In [16]:
previsores

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

#### Existem algumas formas de fazer esse escalonamento, duas bem conhecidas sao a padronizacao e a normalizacao, aqui vamos usar a padronizacao (pesquise mais sobre ambas para entender mais)

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
scaler = StandardScaler()

In [19]:
previsores = scaler.fit_transform(previsores)

In [20]:
previsores

array([[ 1.45393393,  1.36538093,  1.20281942],
       [-0.76217555,  0.5426602 ,  0.69642695],
       [ 0.83682073,  1.67417189,  1.17471147],
       ...,
       [-0.07122592, -0.97448519,  0.35420081],
       [-0.11000289,  1.73936739, -0.92675625],
       [ 1.682986  ,  1.14917639,  0.96381038]])

#### Note que conseguimos padronizar os atributos dos previsores, alem disso nao ha necessidade de padronissar nossa classe. 